# A

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import jax
import jax.numpy as jnp
from flax import nnx
import optax

from src.rts.config import EnvConfig
from src.rts.env import init_state, p1_step
from src.rts.utils import get_legal_moves
from src.rts.visualizaiton import visualize_board
from src.rl.pqn import Params, train_minibatched, Model
from src.rl.eval import evaluate_batch

from carbs import CARBS, CARBSParams, ObservationInParam

In [ ]:
width = 10
height = 10
config = EnvConfig(
    num_players=2,
    board_width = width,
    board_height = height,
    num_neutral_bases = 3,
    num_neutral_troops_start = 5,
    neutral_troops_min = 4,
    neutral_troops_max = 10,
    player_start_troops=5,
    bonus_time=10,
)

In [ ]:
# param_spaces = parse_sweep_config(config.sweep)
# carbs_params = CARBSParams(
#     better_direction_sign=1,
#     is_wandb_logging_enabled=False,
#     resample_frequency=0,
# )
# carbs = CARBS(carbs_params, param_spaces)
num_runs = 1
for _ in range(num_runs):
    # suggestion = carbs.suggest().suggestion

    # config = update_config(config, suggestion)

    params = Params(
        num_iterations=5,
        lr=4e-4,
        gamma=0.99,
        q_lambda=0.92,
        num_envs=50,
        num_steps=250,
        update_epochs=1,
        num_minibatches=4,
        epsilon=0.3,
    )
    q_net = Model(width*height*4, 512, width*height*4, rngs=nnx.Rngs(0))
    optimizer = nnx.Optimizer(q_net, optax.adam(params.lr))
    q_net, losses, cum_returns = train_minibatched(q_net, optimizer, config, params)

    evaluate_batch(q_net, config, jax.random.PRNGKey(0), batch_size=10, num_steps=250)
    # ---

    # obs_out = carbs.observe(ObservationInParam(input=suggestion, output=result, cost=time_used))